In [1]:
#r ".\binaries\BoSSSpad.dll"
#r ".\binaries\XESF.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using ApplicationWithIDT;
using XESF;
Init();


In [2]:
var date = DateTime.Now.ToString("ddMMyyyy");
date

24082023

init workflow


In [3]:
BoSSSshell.WorkflowMgm.Init("XESF_BowShock_ConvStudy_" + date );
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.DefaultDatabase


{ Session Count = 8; Grid Count = 8; Path = \\dc1\userspace\sebastian\cluster\XESF_BowShock_ConvStudy_24082023 }

In [4]:
var dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path;
var database = OpenOrCreateDatabase(dbPath);
var dt = database.Sessions.Last().CreationTime;
dt


2023-08-24 21:32:57Z

In [5]:
var sessions = database.Sessions;
sessions

#0: XDGBowShock_TwoLs	XDGBS-p3-40x128-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo*	08/24/2023 21:37:32	18d0698e...
#1: XDGBowShock_TwoLs	XDGBS-p3-35x112-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo*	08/24/2023 21:37:27	aaec1a4f...
#2: XDGBowShock_TwoLs	XDGBS-p3-30x96-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo*	08/24/2023 21:37:10	45f2a5c0...
#3: XDGBowShock_TwoLs	XDGBS-p3-25x80-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo*	08/24/2023 21:36:50	ffef4ee2...
#4: XDGBowShock_TwoLs	XDGBS-p3-20x64-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo*	08/24/2023 21:34:05	733f7eed...
#5: XDGBowShock_TwoLs	XDGBS-p3-15x48-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo*	08/24/2023 21:33:55	8c7a8863...
#6: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo*	08/24/2023 21:33:46	3ee2a5fd...
#7: XDGBowShock_TwoLs	XDGBS-p3-5x16-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo*	08/24/2023 21:32:57	57118df7...


In [6]:
var si=sessions.Pick(1);

In [7]:
Assembler aaa = new Assembler(3);
var si = database.Sessions.Pick(0);
var ti = si.Timesteps;
si.ID

18d0698e-bf0e-44d2-8f88-6488ae59deb0

## Enthalpy error
enthalpy is computed as
$$h=\frac{(\rho E + p)}{\rho},~ p=(c_P -1)(\rho E - k),~k=c_P \text{Ma}^2\frac{\rho }{2}(\vert u\vert^2 + \vert v\vert^2)$$


For an uniform inviscid flow the inflow enthalpy should remain constant throughout the flowfield 

In the case of the BowShock presented, the v-velocity is zero and we have
$$ h_{in} = \frac{\gamma}{\gamma-1} + \frac{1}{2}u_{in}^2 $$
We prescribe the inflow conditions in terms of pressure $p_{in}=1$, density $\rho_{in}=1$ and the Mach number $M_{in}=4$. From those the velocity is computed as 
$$ u_{in} = M_{in} \sqrt{\gamma \frac{p_{in}}{\rho_{in}}}= 4\sqrt{\gamma}$$

In [8]:
using BoSSS.Solution.CompressibleFlowCommon.MaterialProperty;
var enthalpy_end = (XDGField) si.Timesteps.Last().GetField("h");
var enthalpy_exact = (XDGField) enthalpy_end.Clone();
double gamma = IdealGas.Air.HeatCapacityRatio;
double Ms = 4;
double densityLeft = 1;
double pressureLeft = 1;
double velocityXLeft = Ms * Math.Sqrt(gamma * pressureLeft / densityLeft);

(gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft

14.7

In [9]:
var enthalpy_inflow = (gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft;
enthalpy_exact.Clear();
enthalpy_exact.GetSpeciesShadowField("L").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.GetSpeciesShadowField("R").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.Identification= "h_exact";
var EEN = enthalpy_exact.L2NormAllSpecies();

## Choose which runs to plot

In [30]:
//&&s.Name.Contains("iFlux1")
var sessions = database.Sessions.Where(s => s.Timesteps.Count >0);
//sessions=sessions.Where(s => (s.Name.Contains("iProb0")  &&s.Name.Contains("10x32")&&s.Name.Contains("agg0.2")));
sessions.OrderBy(si => si.GetGrids().Pick(0).NumberOfCells);
sessions


#0: XDGBowShock_TwoLs	XDGBS-p3-40x128-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo*	08/24/2023 21:37:32	18d0698e...
#1: XDGBowShock_TwoLs	XDGBS-p3-35x112-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo*	08/24/2023 21:37:27	aaec1a4f...
#2: XDGBowShock_TwoLs	XDGBS-p3-30x96-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo*	08/24/2023 21:37:10	45f2a5c0...
#3: XDGBowShock_TwoLs	XDGBS-p3-25x80-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo*	08/24/2023 21:36:50	ffef4ee2...
#4: XDGBowShock_TwoLs	XDGBS-p3-20x64-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo*	08/24/2023 21:34:05	733f7eed...
#5: XDGBowShock_TwoLs	XDGBS-p3-15x48-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo*	08/24/2023 21:33:55	8c7a8863...
#6: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo*	08/24/2023 21:33:46	3ee2a5fd...
#7: XDGBowShock_TwoLs	XDGBS-p3-5x16-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo*	08/24/2023 21:32:57	57118df7...


# A Helper Function to obtain different plot formats

In [28]:
using BoSSS.Solution.Tecplot;
using System.IO;
//Directory.GetFiles(".", "*.plt").ForEach(file => File.Delete(file));
var texplot = new Tecplot(enthalpy_exact.GridDat, 2);
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();

public PlotFormat GetFormat(int count){
    
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;

    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}

## Actual Plot


In [12]:
int count=0;
foreach(var si in sessions){
    var enthalpyerrors = new List<double>();
    foreach(var timestep in si.Timesteps){
        var error = ((XDGField) timestep.GetField("h_err")).L2NormAllSpecies()/EEN;
        enthalpyerrors.Add(error);
    }
    allEE.Add(enthalpyerrors);
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Pick(enthalpyerrors.Count -1)).GetInternal();
    plot.AddDataGroup(si.Name,lts.TimeStepNumbers,enthalpyerrors, GetFormat(count));
    count++;
}

plot.Xlabel="Iteration";
plot.LogY = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:1200,yRes:600)

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -4 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 10 1 
 
 
 
 
 10 2 
 
 
 
 
 10 3 
 
 
 
 
 10 4 
 
 
 
 
 0 
 
 
 
 
 20 
 
 
 
 
 40 
 
 
 
 
 60 
 
 
 
 
 80 
 
 
 
 
 100 
 
 
 
 
 120 
 
 
 
 
 140 
 
 
 
 
 160 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 XDGBS-p3-40x128-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo 
 
 
 XDGBS-p3-40x128-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 XDGBS-p3-35x112-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo 
 
 
 XDGBS-p3-35x112-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 XDGBS-p3-30x96-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo 
 
 
 XDGBS-p3-30x96-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 XDGBS-p3-25x80-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo 
 
 
 XDGBS-p3-25x80-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 XDGBS-p3-20x64-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo 
 
 
 XDGBS-p3-20x64-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 XDGBS-p3-15x48-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo 
 
 
 XDGBS-p3-15x48-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M466.8,177.1 L520.2,177.1 M45.6,402.0 L52.5,401.2 L59.4,402.1 L66.3,402.4 L73.2,403.1 L80.1,403.9
 L87.0,404.1 L93.9,403.7 L100.8,406.9 L107.7,408.0 L114.6,65.6 L121.5,410.7 L128.4,411.0 L135.3,411.8
 L142.2,414.8 L149.1,418.0 L156.1,418.5 L163.0,418.6 L169.9,418.4 L176.8,418.3 L183.7,418.4 L190.6,418.3
 L197.5,418.3 L204.4,418.1 L211.3,418.1 L218.2,418.5 L225.1,418.4 L232.0,418.2 L238.9,418.3 L245.8,418.1
 L252.7,418.1 L259.6,428.2 L266.5,432.0 L273.4,440.1 L280.3,440.0 L287.2,442.5 L294.1,453.5 L301.0,484.5
 L307.9,502.7 L314.8,511.4 L321.7,515.0 L328.6,515.3 L335.5,515.6 L342.4,515.4 L349.3,515.6 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 XDGBS-p3-10x32-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo 
 
 
 XDGBS-p3-10x32-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M466.8,201.1 L520.2,201.1 M45.6,407.5 L52.5,400.0 L59.4,401.9 L66.3,402.1 L73.2,402.3 L80.1,402.5
 L87.0,405.4 L93.9,406.4 L100.8,405.4 L107.7,405.9 L114.6,406.5 L121.5,407.5 L128.4,407.7 L135.3,407.7
 L142.2,407.7 L149.1,407.7 L156.1,407.6 L163.0,407.6 L169.9,407.6 L176.8,407.6 L183.7,407.6 L190.6,407.7
 L197.5,407.7 L204.4,407.7 L211.3,407.7 L218.2,407.7 L225.1,407.7 L232.0,407.6 L238.9,407.7 L245.8,407.6
 L252.7,407.6 L259.6,407.6 L266.5,407.7 L273.4,407.7 L280.3,407.7 L287.2,412.0 L294.1,414.0 L301.0,422.1
 L307.9,411.5 L314.8,418.2 L321.7,413.9 L328.6,438.2 L335.5,474.4 L342.4,490.7 L349.3,490.5 L356.2,491.0
 L363.1,490.9 L370.0,491.2 L377.0,491.0 L383.9,491.3 L390.8,490.8 L397.7,491.2 L404.6,490.8 L411.5,491.1
 L418.4,491.1 L425.3,491.1 L432.2,491.0 L439.1,491.1 L446.0,491.1 L452.9,491.1 L459.8,491.1 L466.7,491.0
 L473.6,491.0 L480.5,491.0 L487.4,491.1 L494.3,491.0 L501.2,491.0 L508.1,491.0 L515.0,491.0 L521.9,491.0
 L528.8,491.0 L535.7,491.0 L542.6,491.0 L549.5,491.0 L556.4,491.0 L563.3,491.0 L570.2,491.0 L577.1,491.0
 L584.0,491.0 L590.9,491.0 L597.9,490.9 L604.8,491.0 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 XDGBS-p3-5x16-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo 
 
 
 XDGBS-p3-5x16-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo 
 
 
 
	<path stroke='rgb( 0, 0, 255)

## Residual History

In [32]:

using BoSSS.Solution.Tecplot;
using System.IO;
//Directory.GetFiles(".", "*.plt").ForEach(file => File.Delete(file));
var texplot = new Tecplot(enthalpy_exact.GridDat, 2);
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();
int count=0;
foreach(var si in sessions){
    var enthalpyerrors = new List<double>();
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
    plot.AddDataGroup("||r|| - "+ si.Name,lts.TimeStepNumbers,lts.ResHistory, GetFormat(count));
    count++;
    //plot.AddDataGroup("EnRes_"+si.Name,lts.TimeStepNumbers,lts.EnResHistory, GetFormat(count));
    //count++;
}

plot.Xlabel="Iteration";
plot.LogY = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:2000,yRes:800)

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 10 1 
 
 
 
 
 10 2 
 
 
 
 
 10 3 
 
 
 
 
 0 
 
 
 
 
 20 
 
 
 
 
 40 
 
 
 
 
 60 
 
 
 
 
 80 
 
 
 
 
 100 
 
 
 
 
 120 
 
 
 
 
 140 
 
 
 
 
 160 
 
 
 
 
 180 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 ||r|| - XDGBS-p3-40x128-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo 
 
 
 ||r|| - XDGBS-p3-40x128-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ||r|| - XDGBS-p3-35x112-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo 
 
 
 ||r|| - XDGBS-p3-35x112-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ||r|| - XDGBS-p3-30x96-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo 
 
 
 ||r|| - XDGBS-p3-30x96-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ||r|| - XDGBS-p3-25x80-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo 
 
 
 ||r|| - XDGBS-p3-25x80-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ||r|| - XDGBS-p3-20x64-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo 
 
 
 ||r|| - XDGBS-p3-20x64-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo 
 
 
 
	<path stroke='rgb( 0, 0, 0)' d='M1178.0,153.1 L1231.4,153.1 M45.6,159.0 L56.2,165.7 L66.8,168.4 L77.3,172.3 L87.9,194.1 L98.5,222.1
 L109.1,246.6 L119.7,250.5 L130.2,226.6 L140.8,246.7 L151.4,278.2 L162.0,394.0 L172.6,450.5 L183.1,424.8
 L193.7,498.2 L204.3,517.0 L214.9,506.1 L225.5,523.1 L236.1,517.0 L246.6,530.3 L257.2,528.1 L267.8,538.5
 L278.4,515.5 L289.0,534.1 L299.5,548.3 L310.1,550.8 L320.7,557.4 L331.3,565.9 L341.9,568.6 L352.4,581.9
 L363.0,587.2 L373.6,586.9 L384.2,601.1 L394.8,605.1 L405.3,606.9 L415.9,611.7 L426.5,612.5 L437.1,614.6
 L447.7,615.9 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ||r|| - XDGBS-p3-15x48-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo 
 
 
 ||r|| - XDGBS-p3-15x48-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M1178.0,177.1 L1231.4,177.1 M45.6,151.9 L56.2,152.1 L66.8,167.7 L77.3,170.2 L87.9,174.1 L98.5,184.8
 L109.1,196.7 L119.7,217.9 L130.2,264.9 L140.8,281.7 L151.4,268.2 L162.0,302.9 L172.6,309.1 L183.1,310.1
 L193.7,362.5 L204.3,397.1 L214.9,380.5 L225.5,379.3 L236.1,382.5 L246.6,397.1 L257.2,457.3 L267.8,439.0
 L278.4,453.7 L289.0,450.5 L299.5,438.8 L310.1,431.4 L320.7,435.4 L331.3,422.3 L341.9,421.4 L352.4,450.6
 L363.0,451.1 L373.6,189.9 L384.2,210.7 L394.8,226.0 L405.3,237.5 L415.9,258.6 L426.5,301.1 L437.1,368.6
 L447.7,321.3 L458.2,481.7 L468.8,593.5 L479.4,620.0 L490.0,625.7 L500.6,619.4 L511.1,634.1 L521.7,634.3
 L532.3,632.8 L542.9,651.3 L553.5,655.3 L564.0,648.0 L574.6,660.7 L585.2,667.9 L595.8,679.4 L606.4,682.3
 L617.0,681.2 L627.5,684.0 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ||r|| - XDGBS-p3-10x32-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo 
 
 
 ||r|| - XDGBS-p3-10x32-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M1178.0,201.1 L1231.4,201.1 M45.6,162.1 L56.2,184.8 L66.8,195.5 L77.3,195.9 L87.9,197.1 L98.5,196.0
 L109.1,222.0 L119.7,248.8 L130.2,262.2 L140.8,278.4 L151.4,279.8 L162.0,304.2 L172.6,363.6 L183.1,383.8
 L193.7,403.6 L204.3,421.4 L214.9,400.1 L225.5,428.9 L236.1,427.3 L246.6,464.7 L257.2,397.6 L267.8,469.8
 L278.4,403.6 L289.0,428.9 L299.5,459.2 L310.1,481.1 L320.7,411.3 L331.3,491.0 L341.9,413.4 L352.4,432.5
 L363.0,453.3 L373.6,482.3 L384.2,423.7 L394.8,439.8 L405.3,422.1 L415.9,198.0 L426.5,207.2 L437.1,184.2
 L447.7,236.5 L458.2,276.5 L468.8,264.6 L479

## Convergence Plot Enthalpy

choose the sessions

In [15]:
sessions

#0: XDGBowShock_TwoLs	XDGBS-p3-40x128-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo*	08/24/2023 21:37:32	18d0698e...
#1: XDGBowShock_TwoLs	XDGBS-p3-35x112-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo*	08/24/2023 21:37:27	aaec1a4f...
#2: XDGBowShock_TwoLs	XDGBS-p3-30x96-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo*	08/24/2023 21:37:10	45f2a5c0...
#3: XDGBowShock_TwoLs	XDGBS-p3-25x80-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo*	08/24/2023 21:36:50	ffef4ee2...
#4: XDGBowShock_TwoLs	XDGBS-p3-20x64-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo*	08/24/2023 21:34:05	733f7eed...
#5: XDGBowShock_TwoLs	XDGBS-p3-15x48-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo*	08/24/2023 21:33:55	8c7a8863...
#6: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo*	08/24/2023 21:33:46	3ee2a5fd...
#7: XDGBowShock_TwoLs	XDGBS-p3-5x16-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo*	08/24/2023 21:32:57	57118df7...


they must be reordered for the plot to work...

In [14]:
sessions.OrderBy(si => si.GetGrids().Pick(0).NumberOfCells);
sessions

#0: XDGBowShock_TwoLs	XDGBS-p3-40x128-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo*	08/24/2023 21:37:32	18d0698e...
#1: XDGBowShock_TwoLs	XDGBS-p3-35x112-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo*	08/24/2023 21:37:27	aaec1a4f...
#2: XDGBowShock_TwoLs	XDGBS-p3-30x96-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo*	08/24/2023 21:37:10	45f2a5c0...
#3: XDGBowShock_TwoLs	XDGBS-p3-25x80-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo*	08/24/2023 21:36:50	ffef4ee2...
#4: XDGBowShock_TwoLs	XDGBS-p3-20x64-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo*	08/24/2023 21:34:05	733f7eed...
#5: XDGBowShock_TwoLs	XDGBS-p3-15x48-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo*	08/24/2023 21:33:55	8c7a8863...
#6: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo*	08/24/2023 21:33:46	3ee2a5fd...
#7: XDGBowShock_TwoLs	XDGBS-p3-5x16-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo*	08/24/2023 21:32:57	57118df7...


In [17]:
int[] Cells = sessions.Select(si => si.GetGrids().Pick(0).NumberOfCells).ToArray();
Cells

index,value
0,5120
1,3920
2,2880
3,2000
4,1280
5,720
6,320
7,80


In [19]:
int[] degs = new int[] {0,1,2,3};
var errors= MultidimensionalArray.Create(sessions.Count(),4);


In [20]:
sessions.Pick(2).Timesteps.Last().GetField("rho").Basis.Degree 

0

In [21]:
Assembler aaa = new Assembler(3);
var timesteps = sessions.Pick(2).Timesteps;
var rTSIndex = timesteps.LastIndexWhere(t => t.GetField("rho").Basis.Degree == 2);
rTSIndex

-1

In [33]:
using BoSSS.Solution.CompressibleFlowCommon;
using BoSSS.Solution.CompressibleFlowCommon.MaterialProperty;
var plot = new Plot2Ddata();

public PlotFormat GetFormat(int count){
    
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Circle, PointTypes.OpenCircle, PointTypes.Diamond, PointTypes.OpenDiamond, PointTypes.UpperTriangle, PointTypes.OpenUpperTriangle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue,LineColors.Blue, LineColors.Black, LineColors.Black, LineColors.Red, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointType = PointTypes.OpenCircle;
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;

    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}
string[] STE = new string[] {"L","R"};
var C =XESFHardCodedControl.XDGBowShock_TwoLs_LSFromDB(
        optiLSDegree: 3,
        MaxIterations: 20,
        dgDegreeStart: 0,
        dgDegreeEnd: 0,
        numOfCellsX: 5 * 2,
        numOfCellsY: 10 * 2
    ); 
Material mat = C.GetMaterial();
int count=0;
foreach(int p in degs){
var cells= new List<double>();
var errors= new List<double>();
for(int iSess =0;iSess < sessions.Count();iSess++){
var si = sessions.Pick(iSess);
Console.WriteLine($"P={p},sess={iSess}");
    //extract the relevant field 
    var timesteps = si.Timesteps;
    var rTSIndex = timesteps.LastIndexWhere(t => t.GetField("rho").Basis.Degree == p);
    if(rTSIndex !=-1){

        IDTTimeStepInfo lts = (IDTTimeStepInfo)((TimestepProxy)timesteps.Pick(rTSIndex)).GetInternal();
        
            var fields = lts.Fields.ToList();
            var enthalpyError = (XDGField) lts.GetField("h").Clone();
            SpeciesId[] sIds = new SpeciesId[] {enthalpyError.Basis.Tracker.GetSpeciesId(STE[0]),enthalpyError.Basis.Tracker.GetSpeciesId(STE[1]) };
            enthalpyError.Clear();
            enthalpyError.ProjectFunctionXDG(1.0,
                    (X, U, j) => new StateVector(U, mat).Enthalpy -enthalpy_inflow, 
                    sIds,
                    lts.GetConsFields("iEuler2D"));
    //errors[p,iSess]=(enthalpyError.L2NormAllSpecies()/EEN);
    cells.Add(Math.Sqrt(enthalpyError.Basis.GridDat.iGeomCells.GetCellVolume(0)));
    errors.Add(enthalpyError.L2NormAllSpecies()/EEN);
    }
}
plot.AddDataGroup($"P={p}",cells,errors, GetFormat(count));
count++;
}
plot.Xlabel="Cells";
plot.LogY = true;
plot.LogX = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:1400,yRes:900)

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -4 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
	<path stroke='black' d='M894.2,842.4 L894.2,837.9 M894.2,36.1 L894.2,40.6 M1009.1,842.4 L1009.1,837.9 M1009.1,36.1 L1009.1,40.6
 M1090.5,842.4 L1090.5,837.9 M1090.5,36.1 L1090.5,40.6 M1153.8,842.4 L1153.8,837.9 M1153.8,36.1 L1153.8,40.6
 M1205.4,842.4 L1205.4,837.9 M1205.4,36.1 L1205.4,40.6 M1249.1,842.4 L1249.1,837.9 M1249.1,36.1 L1249.1,40.6
 M1286.9,842.4 L1286.9,837.9 M1286.9,36.1 L1286.9,40.6 M1320.3,842.4 L1320.3,837.9 M1320.3,36.1 L1320.3,40.6
 M1350.1,842.4 L1350.1,833.4 M1350.1,36.1 L1350.1,45.1 '/> 
 10 0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Cells 
 
 
 
 
 P=0 
 
 
 P=0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 P=1 
 
 
 P=1 
 
 
 
 
 
 
 
 
 
 P=2 
 
 
 P=2 
 
 
 
 
 
 
 
 P=3 
 
 
 P=3

## Plotting the ShadowFields (for nice visualization)

choose a concrete session

In [ ]:
var sessions = database.Sessions.Where(s => s.Timesteps.Count >0);
sessions=sessions.Where(s => s.Name.Contains("10x32"));
sessions

In [ ]:
var si = sessions.Pick(5);
si

plot it

In [ ]:
using BoSSS.Solution.Tecplot;
using System.IO;
Directory.GetFiles(".", "*_ShadowPlot_*").ForEach(file => File.Delete(file));
var texplot = new Tecplot(enthalpy_exact.GridDat, 2);
foreach(var timestep in si.Timesteps){
    List<DGField> flds = new List<DGField>();
    var fields = timestep.Fields.ToList();
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) timestep).GetInternal();
    texplot.PlotFields( "XESFBowShock_TwoLs_ShadowPlot_" + lts.TimeStepNumber,0.0,lts.GetShadowFields());
}